In [6]:
import os
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [7]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df



In [8]:
def main():
    #convert to csv, csv files used to create the tfrecords
    for directory in ['train', 'test']:
        image_path = os.path.join(os.getcwd(), '/data/mesonet/labelImg/{}'.format(directory))
        xml_df = xml_to_csv(image_path)
        xml_df.to_csv('/data/mesonet/labelImg/data/{}_labels.csv'.format(directory), index=None)
        print('Successfully converted xml to csv.')




In [9]:
#run main to run this code and create 2 files (test_labels.csv and train_labels.csv)
main()
# ×
# Drag and Drop
# The image will be downloaded

Successfully converted xml to csv.
Successfully converted xml to csv.


In [22]:
#starting tfrecord generation
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

In [23]:
# need to create if for every cat we have
def class_text_to_int(row_label):
    if row_label == 'scattered_clouds':
        return 1
    if row_label == 'clear':
        return 2
    if row_label == 'overcast':
        return 3
    else:
        None

In [24]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [25]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [26]:
#for train
csv_input = "data/mesonet/labelImg/data/train_labels.csv"
output_path = "data/mesonet/labelImg/data/train.record"
image_dir= "/data/mesonet/labelImg/images/"

In [27]:
def main():
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path_new = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path_new))

In [28]:
main()

AttributeError: module 'tensorflow' has no attribute 'python_io'